<a href="https://colab.research.google.com/github/hyonnys/tp2/blob/main/AIB_tp2_Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

2.0.1+cu118 True


In [ ]:
allocated_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # MB
max_allocated_memory = torch.cuda.max_memory_allocated() / (1024 * 1024)  # MB

print(f"Allocated Memory: {allocated_memory:.2f} MB")
print(f"Max Allocated Memory: {max_allocated_memory:.2f} MB")

Allocated Memory: 0.00 MB
Max Allocated Memory: 0.00 MB


In [ ]:
cached_memory = torch.cuda.memory_cached() / (1024 * 1024)  # MB
max_cached_memory = torch.cuda.max_memory_cached() / (1024 * 1024)  # MB

print(f"Cached Memory: {cached_memory:.2f} MB")
print(f"Max Cached Memory: {max_cached_memory:.2f} MB")

Cached Memory: 0.00 MB
Max Cached Memory: 0.00 MB


/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:424: FutureWarning: torch.cuda.max_memory_cached has been renamed to torch.cuda.max_memory_reserved
  warnings.warn(


In [ ]:
!nvidia-smi # A100 GPU

Tue Aug 22 07:29:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 0.Installation

## 0.1 Installing `open-mmlab` framework

* `mmcv`
* `mmengine`
* `mmdetection`

In [ ]:
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0,<2.1.0"

In [ ]:
!pwd

/content


In [ ]:
# Unzip the mmdetection saved with downloaded checkpoint file
import zipfile

ANNO_ZIP = '/content/drive/MyDrive/AIB_tp2/model/mmdetection.zip'

# OUTPUT_DIR = '/content/data/coco'
OUTPUT_DIR = '.'

with zipfile.ZipFile(ANNO_ZIP, 'r') as zip_ref:
  zip_ref.extractall(OUTPUT_DIR)

print("Successfully extracted the zip file.")

!mv /content/content/mmdetection ../
%rm -rf /content/content

Successfully extracted the zip file.


In [ ]:
%cd /content/mmdetection
!pip install -e .

In [ ]:
# Verify the installation
import mmdet
print(mmdet.__version__)

3.1.0


In [ ]:
# Verify the installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

11.8
GCC 9.3


In [ ]:
# Final check on Installation
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet


def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info


if __name__ == '__main__':
    for name, val in collect_env().items():
        print(f'{name}: {val}')

sys.platform: linux
Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
CUDA available: True
numpy_random_seed: 2147483648
GPU 0: NVIDIA A100-SXM4-40GB
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
PyTorch: 2.0.1+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_8

# 1.Preparing the data

* Make `data/coco` directory
* Download the zip files of data
* Unzip them
* Remove the zip files

## 1.1 Create the data directory

In [ ]:
!pwd
%cd /content

/content/mmdetection
/content


In [ ]:
# !mkdir -pv data/coco
!mkdir -pv data/coco

mkdir: created directory 'data'
mkdir: created directory 'data/coco'


## 1.2 Download the data from GoogleDrive

In [ ]:
# Download the annotations.json(COCO file)-labels
import zipfile

ANNO_ZIP = '/content/drive/MyDrive/AIB_tp2/data/annotations.zip'

# OUTPUT_DIR = '/content/data/coco'
OUTPUT_DIR = '/content/data/coco'

with zipfile.ZipFile(ANNO_ZIP, 'r') as zip_ref:
  zip_ref.extractall(OUTPUT_DIR)

print("Successfully extracted the zip file.")

Successfully extracted the zip file.


In [ ]:
# Download the raw dataset
DATA_ZIP = '/content/drive/MyDrive/AIB_tp2/data/dataset.zip'

OUTPUT_DIR = '/content/data/coco'

with zipfile.ZipFile(DATA_ZIP, 'r') as zip_ref:
  zip_ref.extractall(OUTPUT_DIR)

# 2.Modeling with Faster R-CNN

* Not Pre-trained Model : `X-101-64x4d-FPN_1x`
* Pre-trained Model : `R-101-FPN_3x`
* Modeling Environment : `Colab Pro - GPU V100`

※ [Official Github Page](https://github.com/open-mmlab/mmdetection/tree/main/configs/faster_rcnn)

※ [Offical MMDetection Document](https://mmdetection.readthedocs.io/en/latest/user_guides/index.html)

※ Offical Documentation on [Train predefined models on standard datasets](https://mmdetection.readthedocs.io/en/latest/user_guides/train.html)



## 2.1 Download the model

In [ ]:
# Download the faster-rcnn model(checkpoint file ".pth") - Not Pre-trained Model
!pwd
%cd /content/mmdetection/
!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_coco/faster_rcnn_x101_64x4d_fpn_1x_coco_20200204-833ee192.pth\
-O checkpoints/faster_rcnn_x101_64x4d_fpn_1x_coco_20230819.pth

## 2.2 Train a model

- Pre-trained Model of `Faster_rcnn`

In [ ]:
# Download a model - Pretrained Model weights
!pwd
%cd /content/mmdetection/
!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_mstrain_3x_coco/faster_rcnn_r101_fpn_mstrain_3x_coco_20210524_110822-4d4d2ca8.pth\
-O checkpoints/faster_rcnn_r101_fpn_3x_coco_20230820.pth

/content
/content/mmdetection
--2023-08-22 07:15:12--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_mstrain_3x_coco/faster_rcnn_r101_fpn_mstrain_3x_coco_20210524_110822-4d4d2ca8.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.48.85.210, 8.48.85.211, 8.48.85.214, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.48.85.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243519108 (232M) [application/octet-stream]
Saving to: ‘checkpoints/faster_rcnn_r101_fpn_3x_coco_20230820.pth’

checkpoints/faster_ 100%[===================>] 232.24M  28.1MB/s    in 8.4s    

2023-08-22 07:15:20 (27.7 MB/s) - ‘checkpoints/faster_rcnn_r101_fpn_3x_coco_20230820.pth’ saved [243519108/243519108]



In [ ]:
!zip -r /content/drive/MyDrive/AIB_tp2/model/mmdetection.zip ./mmdetection

In [ ]:
!pwd
%cd /content/mmdetection/

/content
/content/mmdetection


In [ ]:
# Create a config file
!cp configs/faster_rcnn/faster-rcnn_r101_fpn_ms-3x_coco.py configs/faster_rcnn/custom_faster-rcnn_r101_fpn_3x_coco.py

In [ ]:
# Modify a config file
config_faster_rcnn_2="""
_base_='faster-rcnn_r101_fpn_ms-3x_coco.py'

# Modify num_classes
model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes=35)))

# Modify dataset related settings
data_root = '/content/data/coco/'
metainfo = {
    'classes': ('인도', '횡단보도', '자전거 도로','교차로', '중앙 차선','안전지대',\
                 '정지선','정지선 위반 판별구역','보행자 신호등 녹색','보행자 신호등 적색',\
                 '차량 신호등 녹색', '차량 신호등 적색',\
                 '오토바이', '오토바이_보행자도로 통행위반', '오토바이_안전모 미착용',\
                 '오토바이_무단횡단','오토바이_신호위반','오토바이_정지선위반',\
                 '오토바이_횡단보도 주행위반','자전거','자전거 캐리어',\
                 '자전거_보행자도로 통행위반', '자전거_안전모 미착용', '자전거_무단횡단',\
                 '자전거_신호위반', '자전거_정지선위반', '자전거_횡단보도 주행위반',\
                 '킥보드', '킥보드 캐리어', '킥보드_보행자도로 통행위반',\
                 '킥보드_안전모 미착용', '킥보드_무단횡단', '킥보드_신호위반',\
                 '킥보드_횡단보도 주행위반', '킥보드_동승자 탑승위반'),
    'palette': [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),\
        (0, 255, 255), (255, 128, 0), (128, 0, 255), (0, 255, 128), (255, 128, 128),\
        (128, 255, 128), (128, 128, 255), (128, 128, 0), (128, 0, 128), (0, 128, 128),\
        (192, 64, 0), (192, 192, 64), (64, 192, 192), (64, 64, 192), (192, 64, 192),\
        (64, 192, 64), (255, 192, 128), (128, 255, 192), (128, 192, 255), (51, 102, 255),\
         (51, 204, 204), (153, 204, 0), (255, 204, 0), (255, 153, 0), (255, 102, 0),\
         (204, 255, 255), (255, 255, 153), (153, 204, 255), (255, 153, 204), (204, 153, 255),\
         (255, 204, 153), (51, 153, 102)
    ]
}
train_dataloader = dict(
    batch_size=16,
    num_workers=4,
    dataset=dict(
      dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/train_annotations.json',
        data_prefix=dict(img='train/'))))
val_dataloader = dict(
    batch_size=16,
    num_workers=4,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/val_annotations.json',
        data_prefix=dict(img='val/')))
test_dataloader = dict(
    batch_size=16,
    num_workers=4,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/test_annotations.json',
        data_prefix=dict(img='test/')))

# Modify metric related settings
val_evaluator = dict(ann_file=data_root + 'annotations/val_annotations.json',
outfile_prefix='./work_dirs/coco_frcnn/val')
test_evaluator = dict(ann_file=data_root + 'annotations/test_annotations.json',
outfile_prefix='./work_dirs/coco_frcnn/test')

# Modify epoch
train_cfg=dict(max_epochs=5)

# Evaluate the model (every epoch ->)every 12 epochs
# train_cfg=dict(val_interval=12)

# Auto_LR_Scaler On
auto_scale_lr = dict(enable=True)

# Use the pre-trained model
load_from = '/content/mmdetection/checkpoints/faster_rcnn_r101_fpn_3x_coco_20230820.pth'
"""

with open('./configs/faster_rcnn/custom_faster-rcnn_r101_fpn_3x_coco.py', 'w') as f:
  f.write(config_faster_rcnn_2)

In [ ]:
# Train a mdoel
# %%time
!python tools/train.py configs/faster_rcnn/custom_faster-rcnn_r101_fpn_3x_coco.py

08/22 22:39:22 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    numpy_random_seed: 1787329042
    GPU 0: NVIDIA A100-SXM4-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;a

In [ ]:
# Save the checkpoint file (.pth) into Google Drive
%cd /content/mmdetection/work_dirs/

!zip -r /content/drive/MyDrive/AIB_tp2/model/faster_rcnn.zip /content/mmdetection/work_dirs

In [ ]:
%cd /content/mmdetection/

!zip -r /content/drive/MyDrive/AIB_tp2/model/mmdetection.zip /content/mmdetection/

## 2.3 Inference and Evaluation

In [ ]:
# Inference with test dataset and save the result
%cd /content/mmdetection/

!python tools/test.py configs/faster_rcnn/custom_faster-rcnn_r101_fpn_3x_coco.py work_dirs/custom_faster-rcnn_r101_fpn_3x_coco/epoch_5.pth\
--out frcnn_results.pkl\
--show-dir frcnn_r101_fpn_3x_results\
--work-dir frcnn_r101_fpn_3x_results_metrics
#checkpoint file work_dirs/ --show-dir faster-rcnn_r101_fpn_3x_results

/content/mmdetection
08/23 04:35:04 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    numpy_random_seed: 972625895
    GPU 0: NVIDIA A100-SXM4-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,c